In [ ]:
import numpy as np, os
import matplotlib.pyplot as plt
import pandas as pd, random
import torch, torch.nn as nn
from PIL import Image
from tqdm import tqdm
from torch.optim import SGD, AdamW
from torch.utils import data
from torchvision import transforms, models
from pydicom import read_file
from pydicom.pixel_data_handlers.util import apply_voi_lut
from sklearn.metrics import classification_report
import multiprocessing
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import SimCLR
from pl_bolts.models.self_supervised.simclr.transforms import (
    SimCLREvalDataTransform, SimCLRTrainDataTransform)

In [ ]:
path = '/scratch1/knarasim/physionet.org/files/vindr-spinexr/1.0.0/'
train_path = path + "train_images/"
test_path  = path + "test_images/"
annot_path = path + "annotations/"
tiny_path = "/".join(path.split("/")[:-2]) + "/tiny_vindr/"

In [ ]:
os.makedirs(tiny_path, exist_ok=True)
os.makedirs(tiny_path + "train_images/", exist_ok=True)
os.makedirs(tiny_path + "test_images/", exist_ok=True)
os.makedirs(tiny_path + "annotations/", exist_ok=True)

def read_xray(path, voi_lut=True, fix_monochrome=True):
    
    dicom = read_file(path)
    # transform raw DICOM data to "human-friendly" view
    if voi_lut and len(dicom.get("VOILUTSequence", [])):
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array.astype("float")
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data -= np.min(data)
    data /= np.max(data)
    data *= 255
        
    return data.astype(np.uint8)

def save_tiny_dicom_train(dicom_file):
    name = dicom_file.split(".")[0]
    if not dicom_file.endswith("dicom"):
        return
    a = read_xray(train_path + dicom_file)
    plt.imsave(tiny_path + f"train_images/{name}.jpg", a)
    
def save_tiny_dicom_test(dicom_file):
    name = dicom_file.split(".")[0]
    if not dicom_file.endswith("dicom"):
        return
    a = read_xray(test_path + dicom_file)
    plt.imsave(tiny_path + f"test_images/{name}.jpg", a)

In [ ]:
#### Make Tiny Version
annotations = pd.read_csv(annot_path + "train.csv")
annotations = annotations[["image_id", "lesion_type", "xmin", "ymin", "xmax", "ymax"]]
annotations.to_csv(tiny_path + "annotations/train.csv")

pool_obj = multiprocessing.Pool(200)
train_dicom_files = os.listdir(train_path)
_ = pool_obj.map(save_tiny_dicom_train,train_dicom_files)
pool_obj.close()

annotations = pd.read_csv(annot_path + "test.csv")
annotations = annotations[["image_id", "lesion_type", "xmin", "ymin", "xmax", "ymax"]]
annotations.to_csv(tiny_path + "annotations/test.csv")

pool_obj = multiprocessing.Pool(200)
test_dicom_files = os.listdir(test_path)
_ = pool_obj.map(save_tiny_dicom_test,test_dicom_files)
pool_obj.close()

In [ ]:
def train_test_split(path):
    train_path = path + "train_images/"
    train_list = [os.path.join(train_path, img) for img in os.listdir(train_path)]
    random.shuffle(train_list)
    threshold = int(0.8 * len(train_list))
    valid_list = train_list[threshold:]
    train_list = train_list[:threshold]
    test_path = path + "test_images/"
    test_list = [os.path.join(test_path, img) for img in os.listdir(test_path)]
    return train_list, valid_list, test_list

In [ ]:
path = '/scratch1/knarasim/physionet.org/files/vindr-spinexr/tiny_vindr/'
train, valid, test = train_test_split(path)
annotations = pd.read_csv(annot_path + "train.csv")
train_annot = [
    annotations.iloc[idx] for idx, row in enumerate(annotations.iterrows()) \
        if "{}train_images/{}.jpg".format(path, row[1]["image_id"]) in train
]
train_annot = pd.DataFrame(train_annot).drop_duplicates(subset=["image_id"]).sort_values(by=["image_id"])
valid_annot = [
    annotations.iloc[idx] for idx, row in enumerate(annotations.iterrows()) \
        if "{}train_images/{}.jpg".format(path, row[1]["image_id"]) in valid
]
valid_annot = pd.DataFrame(valid_annot).drop_duplicates(subset=["image_id"]).sort_values(by=["image_id"])
annotations = pd.read_csv(annot_path + "test.csv")
test_annot = [
    annotations.iloc[idx] for idx, row in enumerate(annotations.iterrows()) \
        if "{}test_images/{}.jpg".format(path, row[1]["image_id"]) in test
]
test_annot = pd.DataFrame(test_annot).drop_duplicates(subset=["image_id"]).sort_values(by=["image_id"])

In [ ]:
class VinDrSpineXR(data.Dataset):
    def __init__(self, root_dir, annot_df, transform=None):
        self.img_paths = root_dir
        self.img_paths.sort()
        self.transform = transform
        self.labels = annot_df[["image_id", "lesion_type"]]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx])
        #  0: Abnormal  |  1: Normal
        label = self.labels.iloc[idx, 1] == "No finding"
        image = self.transform(image)
        return image, label*1
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
batch_size = 32
train_dataset = VinDrSpineXR(train, train_annot, train_transform)
valid_dataset = VinDrSpineXR(valid, valid_annot, train_transform)
test_dataset  = VinDrSpineXR(test , test_annot , test_transform )
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader  = data.DataLoader(test_dataset , batch_size=batch_size, shuffle=True)

In [ ]:
#### Build and Train Network Model
class XRModel(nn.Module):
    def __init__(self, n_classes, use_pretrained=True, freeze=True):
        super(XRModel, self).__init__()
        self.base_net = models.densenet169(pretrained=use_pretrained)
        if freeze:
            for child in self.base_net.children():
                for param in child.parameters():
                    param.requires_grad = False
        n_feats = self.base_net.classifier.in_features
        self.base_net.classifier = nn.Linear(n_feats, 256)
        self.cls = nn.Sequential(nn.LeakyReLU(), nn.Linear(256, n_classes))
    def forward(self, input):
        return self.cls(self.base_net(input))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = XRModel(n_classes=2, freeze=False).to(device)
optim = AdamW(model.parameters(), lr=1e-3)
lossf = nn.CrossEntropyLoss()
n_epochs = 1 #38 + 12

In [ ]:
for i in range(1, n_epochs+1):
    print(f"\nEpoch {i}:")
    print("-"*10)
    running_loss, running_hits = 0.0, 0.0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optim.zero_grad()
        with torch.set_grad_enabled(True):
            out = model(images)
            loss = lossf(out, labels)
            _, pred = torch.max(out, 1)
            loss.backward()
            optim.step()
    
        running_loss += loss.item()
        #running_hits += torch.true_divide(torch.sum(pred == labels),len(labels))
        running_hits += (torch.sum(pred == labels)).item() / batch_size
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc  = running_hits / len(train_loader)
    print("Train Loss: {:.3f}\t Acc: {:.3f}".format(epoch_loss, epoch_acc))
    
    running_loss, running_hits = 0.0, 0.0
    for images, labels in tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            out = model(images)
            loss = lossf(out, labels)
            _, pred = torch.max(out, 1)
    
        running_loss += loss.item()
        #running_hits += torch.true_divide(torch.sum(pred == labels),len(labels))
        running_hits += (torch.sum(pred == labels)).item() / batch_size
        
    epoch_loss = running_loss / len(valid_loader)
    epoch_acc  = running_hits / len(valid_loader)
    print("Valid Loss: {:.3f}\t Acc: {:.3f}".format(epoch_loss, epoch_acc))

In [ ]:
torch.save(model, path + "model.pt")

In [ ]:
predictions, trues = [], []
with torch.set_grad_enabled(False):
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)
        _, pred = torch.max(out, 1)
        predictions.append(pred)
        trues.append(labels)
        
predictions = torch.cat(predictions).detach().cpu().numpy()
trues = torch.cat(trues).detach().cpu().numpy()
print(classification_report(trues, predictions))